# QA4U lecture 1

## what is Quantum Annealing
- QA is mainly used to solve combinatorial optimization problem
- via min or max a cost function
\begin{equation}
E({\bf x})  = \sum_{i=1}^{N} \sum_{j=1}^N Q_{ij} x_i x_j
\end{equation}
- where, $x$ is a binary value whose value is either 1 or 0, $Q_{ij}$ is called QUBO matrix 

## import libs

In [ ]:
# if use d-wave system

token = '**'  # 個人のAPI tokenを使用
endpoint = 'https://cloud.dwavesys.com/sapi/'
from dwave.system import DWaveSampler, EmbeddingComposite
dw_sampler = DWaveSampler(solver='Advantage_system1.1', token=token, endpoint=endpoint)

In [4]:
# if use simulator
from openjij import SQASampler
sim_sampler = SQASampler()

## solve toy prob

### prepare QUBO

In [3]:
import numpy as np

# random number
N = 10
QUBO = np.random.randn(N**2).reshape(N,N)
QUBO

array([[ 0.16734782,  0.49710675,  0.59576211,  0.95632364,  0.60215142,
        -1.26980187,  0.33279204,  0.8441375 , -2.12949381,  0.92841653],
       [-0.89566762,  0.0988476 ,  0.49508946, -2.15349117,  0.29853521,
        -0.40013505, -1.07104299, -0.11723346,  1.53785054, -1.80130759],
       [ 0.5016219 ,  0.03476166,  0.2727773 , -0.12644682, -1.04279514,
        -1.15847408,  0.52075912,  0.22679035, -0.48904818,  1.6068664 ],
       [-0.0880333 ,  0.12135502, -0.30288433,  1.15921598, -0.79015184,
        -1.19638479, -0.18916064, -0.40107717,  0.72556785, -0.10215349],
       [ 0.36910011,  0.07665527,  1.19220491, -1.31071732, -0.78039642,
         0.06382301,  0.83891665, -2.09700236,  1.71045874,  0.78427488],
       [ 1.83791438, -0.02414485,  2.53737923, -0.20210626,  0.0882964 ,
        -0.48853465, -0.40063011,  0.31897304, -1.01572602, -0.98791084],
       [ 1.35375925, -1.08227533,  0.05886835,  0.11621282,  0.16101261,
         0.6826371 , -1.31955956, -2.20491373

### solve prob

In [ ]:
# if use d-wave
sampler = EmbeddingComposite(dw_sampler)
sampleset = sampler.sample_qubo(QUBO, num_reads=10)
sampleset.record

In [16]:
# if use simulator
QUBOdict = {}
for i in range(N):
    for j in range(N):
        QUBOdict[(i,j)] = QUBO[i][j]

# input of QUBO for openjij simulator must be dictionary type
sampleset = sim_sampler.sample_qubo(QUBOdict, num_reads=10)
print(sampleset.record)
print(sampleset.first)

[([0, 1, 0, 1, 1, 1, 1, 1, 0, 1], -16.32472029, 1)
 ([0, 1, 0, 1, 1, 1, 1, 1, 0, 1], -16.32472029, 1)
 ([0, 1, 0, 1, 1, 1, 1, 1, 0, 1], -16.32472029, 1)
 ([0, 1, 0, 1, 1, 1, 1, 1, 0, 1], -16.32472029, 1)
 ([0, 1, 0, 1, 1, 1, 1, 1, 0, 1], -16.32472029, 1)
 ([0, 1, 0, 1, 1, 1, 1, 1, 0, 1], -16.32472029, 1)
 ([0, 1, 0, 1, 1, 1, 1, 1, 0, 1], -16.32472029, 1)
 ([0, 1, 0, 1, 1, 1, 1, 1, 0, 1], -16.32472029, 1)
 ([0, 1, 0, 1, 1, 1, 1, 1, 0, 1], -16.32472029, 1)
 ([0, 1, 0, 1, 1, 1, 1, 1, 0, 1], -16.32472029, 1)]
Sample(sample={0: 0, 1: 1, 2: 0, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 0, 9: 1}, energy=-16.324720288882094, num_occurrences=1)


## solve knapsack problem
problem assumption
- two people, N objects
- total weight $W = \sum_{i=1}^N w_i$
    - weights for person A $W_A = \sum_{i=1}^N w_i x_i$, if A picks $i$th object, $x_i=0$
    - weights for person B $W_B = W - W_A$
- purpose, to make $W_A - W_B = 0$
    - cost function 
    \begin{align}
    E(x) &= (W_A - W_B)^2 = (2W - W_A)\\
    &= (2\sum_{i=1}^N w_i x_i - W)^2\\
    &= (2\sum_{i=1}^N w_i x_i - W) (2\sum_{j=1}^N w_j x_j - W)\\
    &= 4\sum_{i=1}^N\sum_{j=1}^N w_iw_j x_ix_j - 2W\sum_{i=1}^N w_i x_i  - 2W\sum_{j=1}^N w_j x_j + W^2\\
    &= 4\sum_{i=1}^N\sum_{j=1}^N w_iw_j x_ix_j - 4W\sum_{i=1}^N w_i x_i + W^2
    \end{align}
- equivalent QUBO $4w_i w_j$

In [9]:
# assume 10 objects with random weights
N = 10
w = np.random.rand(N)

# total weights
W = np.sum(w)

In [17]:
# prepare QUBO
Q = np.zeros(N**2).reshape(N,N)

# due to first item in E(x) func
for i in range(N):
  for j in range(N):
    Q[i][j] = 4 * w[i] * w[j]


# due to second item
for i in range(N):
  Q[i][i] = Q[i][i] - 4 * W * w[i]

# convert to dictionary format
Qdict = {}
for i in range(N):
  for j in range(N):
      Qdict[(i,j)] = Q[i][j]

In [18]:
# if use simulator
sampleset = sim_sampler.sample_qubo(Qdict, num_reads=10)
sampleset.record

rec.array([([0, 1, 0, 1, 0, 1, 0, 1, 0, 1], -19.87803716, 1),
           ([0, 1, 0, 1, 1, 1, 0, 0, 0, 1], -20.00051313, 1),
           ([0, 1, 1, 0, 1, 1, 1, 1, 0, 0], -19.61849634, 1),
           ([1, 0, 1, 0, 1, 0, 0, 0, 1, 0], -19.96568743, 1),
           ([0, 0, 0, 1, 1, 0, 1, 0, 0, 1], -19.96281633, 1),
           ([0, 0, 1, 0, 0, 0, 0, 1, 1, 1], -19.77070938, 1),
           ([0, 1, 0, 0, 1, 1, 1, 1, 1, 0], -19.99259101, 1),
           ([1, 0, 1, 0, 0, 1, 1, 1, 0, 0], -20.00164098, 1),
           ([1, 0, 0, 0, 1, 0, 0, 0, 1, 1], -20.0184982 , 1),
           ([0, 1, 1, 1, 1, 1, 0, 0, 1, 0], -19.93549473, 1)],
          dtype=[('sample', 'i1', (10,)), ('energy', '<f8'), ('num_occurrences', '<i8')])